In [1]:
import pyterrier as pt
import pandas as pd
from tira.third_party_integrations import ensure_pyterrier_is_loaded
from tira.rest_api_client import Client
from ir_measures import nDCG

pd.set_option('display.width', 1000)

ensure_pyterrier_is_loaded()
tira = Client()

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:
DATASETS = {
    # 'longeval-train-20230513-training': 'train_2023', 
    # 'longeval-heldout-20230513-training': 'WT',
    # 'longeval-short-july-20230513-training': 'ST',
    # 'longeval-long-september-20230513-training': 'LT',
    'longeval-2023-01-20240423-training': 'train_2024',
    # 'longeval-2023-06-20240418-training': 'lag6',
    # 'longeval-2023-08-20240418-training': 'lag8',
}

In [3]:
for dataset_name, run_suffix in DATASETS.items():
    dataset = pt.get_dataset(f'irds:ir-benchmarks/{dataset_name}')

    bm25 = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/BM25 Re-Rank (tira-ir-starter-pyterrier)', dataset)
    ltr = pt.io.read_results(f'../runs/ows_ltr/ows_ltr.{run_suffix}')

    print(dataset_name)
    print(pt.Experiment(
        [bm25, ltr],
        # dataset.get_topics(variant='query'),
        # dataset.get_qrels(variant='relevance'),
        pd.read_csv('../splits/2024/topics_val.csv'),
        pd.read_csv('../splits/2024/qrels_val.csv'),
        eval_metrics=[nDCG@10, nDCG, nDCG(judged_only=True)@10, nDCG(judged_only=True)],
        names=['BM25', 'BM25 + LMART']
    ).round(3))
    print()

longeval-2023-01-20240423-training
           name  nDCG@10   nDCG  nDCG(judged_only=True)@10  nDCG(judged_only=True)
0          BM25    0.197  0.319                      0.448                   0.482
1  BM25 + LMART    0.284  0.379                      0.483                   0.502

